In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model
from pathlib import Path
import kagglehub
from sklearn.metrics.pairwise import cosine_similarity

/Users/apurbakoirala/anaconda3/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
# path = kagglehub.dataset_download("cubeai/dog-nose-detection-for-yolov8")

# print("Path to dataset files:", path)

In [3]:
!ls

Arcface DogNose.ipynb  Prediction Model.ipynb content


In [4]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EMBEDDING_SIZE = 256
NUM_CLASSES = 100 

In [5]:
def load_dataset(data_path):
    return tf.keras.utils.image_dataset_from_directory(
        data_path,
        label_mode='int',
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True
    )

train_ds = load_dataset('content/train')
val_ds = load_dataset('content/valid')

Found 2916 files belonging to 2 classes.
Found 184 files belonging to 2 classes.


2025-03-13 14:16:32.081088: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-03-13 14:16:32.081116: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-03-13 14:16:32.081119: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1741855592.081484 9818873 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1741855592.081513 9818873 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomContrast(0.2),
    layers.RandomZoom(0.2)
])

In [7]:
# Preprocessing
normalize = layers.Rescaling(1./127.5, offset=-1)  # MobileNetV3 scaling

In [8]:
def preprocess(image, label):
    image = augmentation(image)
    image = normalize(image)
    label = tf.cast(label, tf.int32)  # Convert labels to int32
    return image, label

In [9]:
train_ds = train_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)

In [10]:
class ArcFace(layers.Layer):
    def __init__(self, n_classes, s=30.0, m=0.50, **kwargs):
        super().__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.n_classes),
            initializer='glorot_uniform',
            trainable=True,
            name='arcface_weights'
        )
        
    def call(self, inputs, labels=None):
        # Normalize features and weights
        x_norm = tf.nn.l2_normalize(inputs, axis=1)
        w_norm = tf.nn.l2_normalize(self.w, axis=0)
        
        # Calculate cosine similarity
        cos_theta = tf.matmul(x_norm, w_norm)
        
        if labels is not None:
            # Convert labels to one-hot
            one_hot = tf.one_hot(labels, depth=self.n_classes)
            
            # Calculate theta + margin
            theta = tf.acos(tf.clip_by_value(cos_theta, -1.0 + 1e-7, 1.0 - 1e-7))
            margin_theta = theta + self.m
            cos_margin = tf.cos(margin_theta)
            
            # Apply margin to correct class
            final_logits = self.s * (one_hot * cos_margin + (1 - one_hot) * cos_theta)
            return final_logits
        
        return self.s * cos_theta

In [11]:
def build_model():
    # MobileNetV3 backbone
    base_model = tf.keras.applications.MobileNetV3Small(
        input_shape=(*IMG_SIZE, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg'
    )
    
    # Freeze base layers
    base_model.trainable = False
    
    # Embedding layer
    inputs = layers.Input(shape=(*IMG_SIZE, 3))
    x = base_model(inputs)
    embeddings = layers.Dense(EMBEDDING_SIZE)(x)
    
    # ArcFace classification head
    arcface = ArcFace(n_classes=NUM_CLASSES, name='arc_face')(embeddings)
    
    return Model(inputs, arcface)  # Single output during training

In [12]:
model = build_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobileNetV3Small (Functional)   │ (None, 576)            │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ arc_face (ArcFace)              │ (None, 100)            │        25,600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,112,432 (4.24 MB)

 Trainable params: 173,312 (677.00 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [13]:
class ArcFaceLoss(tf.keras.losses.Loss):
    def __init__(self, n_classes, s=30.0, m=0.50, **kwargs):
        super().__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ce = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE
        )
        
    def call(self, y_true, y_pred):
        # Convert y_true to int32
        y_true = tf.cast(y_true, tf.int32)
        
        # Convert labels to one-hot
        one_hot = tf.one_hot(y_true, depth=self.n_classes)
        
        # Calculate theta + margin
        cos_theta = y_pred / self.s  # Reverse scaling
        theta = tf.acos(tf.clip_by_value(cos_theta, -1.0 + 1e-7, 1.0 - 1e-7))
        margin_theta = theta + self.m
        cos_margin = tf.cos(margin_theta)
        
        # Apply margin to the correct class
        adjusted_logits = self.s * (one_hot * cos_margin + (1 - one_hot) * cos_theta)
        
        # Compute loss
        return self.ce(y_true, adjusted_logits)

In [14]:
# Pass NUM_CLASSES (100) to ArcFaceLoss
losses = {
    'arc_face': ArcFaceLoss(n_classes=NUM_CLASSES),  # Match the output name
}

In [15]:
# In your model compilation:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=losses,
    metrics={'arc_face': ['accuracy']}
)

In [16]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)
]

In [17]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5


2025-03-13 14:16:33.055238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 14s 130ms/step - accuracy: 0.9422 - loss: 2.6092 - val_accuracy: 1.0000 - val_loss: 2.8118e-07 - learning_rate: 0.0010
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 115ms/step - accuracy: 1.0000 - loss: 2.6720e-07 - val_accuracy: 1.0000 - val_loss: 2.7340e-07 - learning_rate: 0.0010
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 1.0000 - loss: 8.2026e-07 - val_accuracy: 1.0000 - val_loss: 2.9867e-07 - learning_rate: 0.0010
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 115ms/step - accuracy: 1.0000 - loss: 2.6151e-07 - val_accuracy: 1.0000 - val_loss: 2.7081e-07 - learning_rate: 5.0000e-04
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 117ms/step - accuracy: 1.0000 - loss: 2.6480e-07 - val_accuracy: 1.0000 - val_loss: 2.7081e-07 - learning_rate: 5.0000e-04
